In [1]:
import tensorflow as tf

In [ ]:
# Assume that we are receiving a batch of 40x40 pixel patches
# The associated set of labels indicates whether or not a patch contains a crack
# These labels are one-hot encoded.

patch_size  = 40
input_size  = patch_size**2
batch_size  = 32
in_channels = 1

graph = tf.Graph()

with graph.as_default():
    # Inputs
    images = tf.placeholder(tf.float32, [batch_size, input_size, in_channels])
    labels = tf.placeholder(tf.float32, [batch_size, 2])
    
    # Variables
    mu    = tf.Variable(tf.truncated_normal(
                            shape = []))
    Sigma = 
    Phi   =
    
    Eh    =
    Ehh   = 